In [1]:
import numpy as np

X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],   # для умножения на intercept
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2]]) # стаж

# средний балл ЕГЭ (целевая переменная)
y = [45, 55, 50, 59, 65, 35, 75, 80, 50, 60]

In [2]:
def calc_mse(y, y_pred):
    err = np.mean((y - y_pred)**2)
    return err

def calc_mae(y, y_pred):
    err = np.mean(np.abs(y - y_pred))
    return err

1. Подберите скорость обучения (alpha) и количество итераций:

In [3]:
def func(n, w, alpha, n_iter):
    for i in range(n_iter):
        y_pred = np.dot(w, X)
        err = calc_mse(y, y_pred)
        for j in range(w.shape[0]):
            w[j] -= alpha * (1/n * 2 * np.sum(X[j] * (y_pred - y)))
        if i % 100 == 0:
            print(i, w, err)

n = X.shape[1]
w = np.array([1, 0.5])

i = 1
for alpha in [1e-1, 1e-2, 1e-3]:
    for n_iter in [500, 800, 1000, 1200]:
        print(f'\nПопытка #{i}: alpha={alpha}, n_iter={n_iter}')
        func(X.shape[1], np.array([1, 0.5]), alpha, n_iter)
        i += 1


Попытка #1: alpha=0.1, n_iter=500
0 [12.02 34.5 ] 3173.15
100 [1.21092169e+31 6.55765787e+31] 1.6499598531554106e+64
200 [3.40912000e+61 1.84618401e+62] 1.3077533388173627e+125
300 [9.59772979e+91 5.19758039e+92] 1.0365214595478304e+186
400 [2.70205851e+122 1.46328003e+123] 8.215438678021257e+246

Попытка #2: alpha=0.1, n_iter=800
0 [12.02 34.5 ] 3173.15
100 [1.21092169e+31 6.55765787e+31] 1.6499598531554106e+64
200 [3.40912000e+61 1.84618401e+62] 1.3077533388173627e+125
300 [9.59772979e+91 5.19758039e+92] 1.0365214595478304e+186
400 [2.70205851e+122 1.46328003e+123] 8.215438678021257e+246
500 [7.60713247e+152 4.11958697e+153] inf
600 [2.14164365e+183 1.15979146e+184] inf
700 [6.02939088e+213 3.26517254e+214] inf

Попытка #3: alpha=0.1, n_iter=1000
0 [12.02 34.5 ] 3173.15
100 [1.21092169e+31 6.55765787e+31] 1.6499598531554106e+64
200 [3.40912000e+61 1.84618401e+62] 1.3077533388173627e+125
300 [9.59772979e+91 5.19758039e+92] 1.0365214595478304e+186
400 [2.70205851e+122 1.46328003e+123]

### Оптимальна попытка #3: alpha=0.01, n_iter=1000

*2. В этом коде мы избавляемся от итераций по весам, но тут есть ошибка, исправьте ее:

In [4]:
n = X.shape[1]
alpha = 1e-2
w = np.array([1, 0.5])

for i in range(1000):
    y_pred = np.dot(w, X)
    err = calc_mse(y, y_pred)
    '''for j in range(W.shape[0]):
        W[j] -= alpha * (1/n * 2 * np.sum(X[j] * (y_pred - y)))'''
    # w -= (alpha * (1/n * 2 * np.sum(X * (y_pred - y))))
    w -= (alpha * (1/n * 2 * np.dot(X, (np.dot(w, X) - y))))
    if i % 100 == 0:
        print(i, w, err)

0 [2.102 3.9  ] 3173.15
100 [31.88770806  6.74418155] 175.19445858001842
200 [41.83683774  4.90699865] 61.9177717428135
300 [45.33508261  4.26102097] 47.913169919666785
400 [46.56511152  4.03388672] 46.181755648107604
500 [46.99760587  3.95402334] 45.96769776787538
600 [47.14967657  3.92594232] 45.941233404700036
700 [47.20314662  3.91606866] 45.93796156758051
800 [47.2219474   3.91259695] 45.93755706443538
900 [47.228558    3.91137626] 45.937507054979434


*3. Вместо того, чтобы задавать количество итераций, задайте условие остановки алгоритма - когда ошибка за итерацию начинает изменяться ниже определенного порога

In [5]:
n = X.shape[1]
alpha = 1e-2
w = np.array([1, 0.5])
threshold = 1e-10

i = 0
err_prev = 0
while True:
    y_pred = np.dot(w, X)
    err = calc_mse(y, y_pred)
    w -= (alpha * (1/n * 2 * np.dot(X, (np.dot(w, X) - y))))
    if i % 100 == 0:
        print(i, w, err)
    
    if abs(err_prev - err) < threshold or i > 10000:
        break
    i += 1
    err_prev = err

0 [2.102 3.9  ] 3173.15
100 [31.88770806  6.74418155] 175.19445858001842
200 [41.83683774  4.90699865] 61.9177717428135
300 [45.33508261  4.26102097] 47.913169919666785
400 [46.56511152  4.03388672] 46.181755648107604
500 [46.99760587  3.95402334] 45.96769776787538
600 [47.14967657  3.92594232] 45.941233404700036
700 [47.20314662  3.91606866] 45.93796156758051
800 [47.2219474   3.91259695] 45.93755706443538
900 [47.228558    3.91137626] 45.937507054979434
1000 [47.23088237  3.91094704] 45.937500872219864
1100 [47.23169965  3.91079613] 45.93750010783411
1200 [47.23198702  3.91074306] 45.93750001333172
